# Sebuah notebook copy dari Mr. Abishek Takur untuk pendekatan masalah NLP apa saja

example data dari kompetisi Spooky author Identification

Notebook ini akn membuat sebuah basic model dari Deep Learning, selain itu diakhir ini juga akan diberikan sedikit pencerahan tentang gambaran umum <i>ensembling</i>


### Cakupan nya ialah:
- tfidf 
- count features
- logistic regression
- naive bayes
- svm
- xgboost
- grid search
- word vectors
- LSTM
- GRU
- Ensembling

## 1. Import Library

In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM,GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPool1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

## 2. Import Data

In [11]:
import os

print(os.getcwd())

g:\VSCode Repo\Kaggle\2 - Spooky Author Identification


In [14]:
test_data = pd.read_csv('spooky-author-identification/test.csv')
test_data.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [15]:
train_data = pd.read_csv('spooky-author-identification/train.csv')
train_data.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [16]:
sample = pd.read_csv('spooky-author-identification/sample_submission.csv')
sample.head()

,id,EAP,HPL,MWS
0,id02310,0.403494,0.287808,0.308698
1,id24541,0.403494,0.287808,0.308698
2,id00134,0.403494,0.287808,0.308698
3,id27757,0.403494,0.287808,0.308698
4,id04081,0.403494,0.287808,0.308698


Output prediksi author EAP atau HPL ataupun MWS. Selain itu metriks pengukuran menggunakan <i>Multi-Class Logarithmic Loss</i>

In [18]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """ Multiclass Version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class prediction, one probability per classes
    """
    #Konversi matriks aktual ke binary array bila belum
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i,val in enumerate(actual):
            actual2[i, val]=1
        actual = actual2
        
    clip = np.clip(predicted, eps, 1-eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota    

Menggunakan LabelEncoders untuk mengonversi classes kedalam value integers 0,1, dan 2

In [19]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train_data.author.values)
y

array([0, 1, 0, ..., 0, 0, 1])

Split train dan test data

In [26]:
xtrain, xtest, ytrain, ytest = train_test_split( train_data.text.values, y, 
                                                stratify=y,
                                                random_state= 42,
                                                test_size=0.1 , shuffle=True,
                                                )
print(xtrain.shape)
print(xtest.shape)

(17621,)
(1958,)


## Building Basic Model